In [ ]:
import geemap
import datetime
from google.colab import drive
# drive.mount('/content/drive')
import ee
ee.Authenticate()
ee.Initialize(project='ee-thaarushiherath')

aoi = ee.Geometry.Polygon([
    [
        [79.80086, 7.62606],
        [79.86712, 7.62470],
        [79.86712, 7.59595],
        [79.78592, 7.59731]
    ]
])


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [ ]:
# Define date range and area of interest (replace `aoi` with your geometry)
start_date = '2024-06-01'
end_date = '2024-08-30'

# Load the Landsat 8 ImageCollection
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER')

# Function to mask clouds using the QA_PIXEL band (specific for Landsat 8)
def cloud_mask(image):
    cloud_shadow_bit_mask = (1 << 3)  # Cloud shadow bit
    clouds_bit_mask = (1 << 5)       # Clouds bit
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0).And(
           qa.bitwiseAnd(clouds_bit_mask).eq(0))
    return image.updateMask(mask)

# Function to export a single image with NDVI, NDWI, and MNDWI
def export_image(image):
    # Apply cloud mask
    image = cloud_mask(image)

    # Extract acquisition date as a string
    acquisition_date = ee.Date(image.get('system:time_start')).format('YYYY_MM_dd').getInfo()

    # Calculate NDVI, NDWI, and MNDWI
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
    mndwi = image.normalizedDifference(['SR_B3', 'SR_B6']).rename('MNDWI')

    # Define the Google Drive folder for exports
    drive_folder = 'GEE_2024'

    # Export NDVI to Google Drive
    ee.batch.Export.image.toDrive(
        image=ndvi,
        description=f'Export_NDVI_{acquisition_date}',
        folder=drive_folder,
        fileNamePrefix=f'NDVI_{acquisition_date}',
        scale=30,
        region=aoi,
        crs='EPSG:4326'
    ).start()

    # Export NDWI to Google Drive
    ee.batch.Export.image.toDrive(
        image=ndwi,
        description=f'Export_NDWI_{acquisition_date}',
        folder=drive_folder,
        fileNamePrefix=f'NDWI_{acquisition_date}',
        scale=30,
        region=aoi,
        crs='EPSG:4326'
    ).start()

    # Export MNDWI to Google Drive
    ee.batch.Export.image.toDrive(
        image=mndwi,
        description=f'Export_MNDWI_{acquisition_date}',
        folder=drive_folder,
        fileNamePrefix=f'MNDWI_{acquisition_date}',
        scale=30,
        region=aoi,
        crs='EPSG:4326'
    ).start()

# Convert the ImageCollection to a list for iterative processing
image_list = landsat.toList(landsat.size())

# Process each image in the list
num_images = image_list.size().getInfo()
if num_images > 0:
    for i in range(num_images):
        image = ee.Image(image_list.get(i))
        export_image(image)
    print(f"Export tasks initiated for {num_images} images.")
else:
    print("No images found for the specified date range and location.")


Export tasks initiated for 9 images.


In [ ]:

start_date = '1988-01-01'
end_date = '2012-12-30'


landsat = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER')

def cloud_mask(image):
    cloud_shadow_bit_mask = (1 << 4)
    clouds_bit_mask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0).And(
           qa.bitwiseAnd(clouds_bit_mask).eq(0))
    return image.updateMask(mask)

def export_image(image):
    image = cloud_mask(image)
    acquisition_date = ee.Date(image.get('system:time_start')).format('YYYY_MM_dd').getInfo()
    ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')  # NIR, Red
    ndwi = image.normalizedDifference(['SR_B2', 'SR_B4']).rename('NDWI')  # Green, NIR
    mndwi = image.normalizedDifference(['SR_B2', 'SR_B5']).rename('MNDWI')  # Green, SWIR1
    drive_folder = 'GEE_Landsat5_Exports'

    ee.batch.Export.image.toDrive(
        image=ndvi,
        description=f'Export_NDVI_{acquisition_date}',
        folder=drive_folder,
        fileNamePrefix=f'NDVI_{acquisition_date}',
        scale=30,
        region=aoi,
        crs='EPSG:4326'
    ).start()

    ee.batch.Export.image.toDrive(
        image=ndwi,
        description=f'Export_NDWI_{acquisition_date}',
        folder=drive_folder,
        fileNamePrefix=f'NDWI_{acquisition_date}',
        scale=30,
        region=aoi,
        crs='EPSG:4326'
    ).start()

    ee.batch.Export.image.toDrive(
        image=mndwi,
        description=f'Export_MNDWI_{acquisition_date}',
        folder=drive_folder,
        fileNamePrefix=f'MNDWI_{acquisition_date}',
        scale=30,
        region=aoi,
        crs='EPSG:4326'
    ).start()

image_list = landsat.toList(landsat.size())

num_images = image_list.size().getInfo()
if num_images > 0:
    for i in range(num_images):
        image = ee.Image(image_list.get(i))
        export_image(image)
    print(f"Export tasks initiated for {num_images} images.")
else:
    print("No images found for the specified date range and location.")


Export tasks initiated for 301 images.


In [ ]:

# Get a list of all tasks
tasks = ee.batch.Task.list()

# Iterate through the tasks and cancel them
for task in tasks:
    if task.state in ['RUNNING', 'READY']:
        print(f"Cancelling task: {task.id}, Name: {task.config.get('description', 'No description')}")
        task.cancel()

In [ ]:
# check availability
start_date = '2001-01-01'
end_date = '2002-12-30'

# Landsat 5 collection
landsat5_collection = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date)

# Get count of images in this date range
count = landsat5_collection.size().getInfo()
print(f"Number of images from {start_date} to {end_date}: {count}")

Number of images from 2001-01-01 to 2002-12-30: 5


In [ ]:
# check availability
start_date = '2002-01-01'
end_date = '2012-12-30'

# Landsat 5 collection
landsat5_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date)

# Get count of images in this date range
count = landsat5_collection.size().getInfo()
print(f"Number of images from {start_date} to {end_date}: {count}")

In [ ]:
start_date = '2024-10-01'
end_date = '2024-12-30'
#landsat 8 (2013 - present)
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .sort('CLOUD_COVER') \
            .first()

# landsat 5 (1982 - 2013)
# landsat = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
#             .filterBounds(aoi) \
#             .filterDate(start_date, end_date) \
#             .sort('CLOUD_COVER') \
#             .first()


band_sr_b3 = landsat.select(['SR_B3']) #this is only needed for LANDSAT8
band_sr_b5 = landsat.select(['SR_B5'])
band_sr_b6 = landsat.select(['SR_B6']) #SR_B6 is only available for LANDSAT8
# band_sr_b2 = landsat.select(['SR_B2']) #for LANDSAT5
band_sr_b4 = landsat.select(['SR_B4'])

"""individual band download fir landsat 8"""

if landsat is None:
    print("No images found for the specified date range and location.")
else:
    acquisition_date = ee.Date(landsat.get('system:time_start')).format('YYYY_MM_dd').getInfo()
    print(f"Image acquisition date: {acquisition_date}")



# out_dir_b3 = 'SR_B3_output.tif'
# out_dir_b5 = 'SR_B5_output.tif'
# out_dir_b6 = 'SR_B6_output.tif'
# out_dir_b4 = 'SR_B4_output.tif'


# geemap.ee_export_image(band_sr_b3, filename=out_dir_b3, scale=30, region=aoi, file_per_band=False)
# geemap.ee_export_image(band_sr_b5, filename=out_dir_b5, scale=30, region=aoi, file_per_band=False)
# geemap.ee_export_image(band_sr_b6, filename=out_dir_b6, scale=30, region=aoi, file_per_band=False)
# geemap.ee_export_image(band_sr_b4, filename=out_dir_b4, scale=30, region=aoi, file_per_band=False)


# print(f"Band SR_B3 saved as {out_dir_b3}")
# print(f"Band SR_B5 saved as {out_dir_b5}")
# print(f"Band SR_B6 saved as {out_dir_b6}")
# print(f"Band SR_B4 saved as {out_dir_b4}")


Image acquisition date: 2024_11_07


In [ ]:
# landsat 8 index calculation

ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
ndwi = landsat.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
mndwi = landsat.normalizedDifference(['SR_B3', 'SR_B6']).rename('MNDWI')

In [ ]:
#  landsat 5 index calculation

ndvi = landsat.normalizedDifference(['SR_B4', 'SR_B2']).rename('NDVI')
ndwi = landsat.normalizedDifference(['SR_B2', 'SR_B4']).rename('NDWI')
mndwi = landsat.normalizedDifference(['SR_B2', 'SR_B5']).rename('MNDWI')

In [ ]:
drive_folder='GEE_Exports'

ee.batch.Export.image.toDrive(
        image=ndvi,
        description=f'Export_NDVI_{acquisition_date}',
        folder=drive_folder,
        fileNamePrefix=f'NDVI_{acquisition_date}',
        scale=30,
        region=aoi,
        crs='EPSG:4326'
    ).start()

    # Export NDWI to Google Drive
ee.batch.Export.image.toDrive(
        image=ndwi,
        description=f'Export_NDWI_{acquisition_date}',
        folder=drive_folder,
        fileNamePrefix=f'NDWI_{acquisition_date}',
        scale=30,
        region=aoi,
        crs='EPSG:4326'
    ).start()

ee.batch.Export.image.toDrive(
        image=mndwi,
        description=f'Export_MNDWI_{acquisition_date}',
        folder=drive_folder,
        fileNamePrefix=f'MNDWI_{acquisition_date}',
        scale=30,
        region=aoi,
        crs='EPSG:4326'
    ).start()
print(f"Export tasks for NDVI, NDWI, and MNDWI submitted to Google Drive folder: {drive_folder}.")

Export tasks for NDVI, NDWI, and MNDWI submitted to Google Drive folder: GEE_Exports_Indice.


In [ ]:
out_dir_ndvi = 'ndvi.tif'
out_dir_ndwi = 'ndwi.tif'
out_dir_mndwi = 'mndwi.tif'

geemap.ee_export_image(ndvi, filename=out_dir_ndvi, scale=30, region=aoi, file_per_band=False)
geemap.ee_export_image(ndwi, filename=out_dir_ndwi, scale=30, region=aoi, file_per_band=False)
geemap.ee_export_image(mndwi, filename=out_dir_mndwi, scale=30, region=aoi, file_per_band=False)


Generating URL ...
Please wait ...
Data downloaded to /content/ndvi.tif
Generating URL ...
Please wait ...
Data downloaded to /content/ndwi.tif
Generating URL ...
Please wait ...
Data downloaded to /content/mndwi.tif


0
